In [3]:
import numpy as np
import pandas as pd
import scipy.stats as ss

## 目标: 用来判断由一个名义变量分割的多个总体的均值是不是相等
  - 方差分析(analysis of variance)，简写为ANOVA，指的是利用对多个样本的方差的分析，得出总体均值是否相等的判定
  
1.假定条件
  - 每个样本的值服从正态分布
  - 每个样本的方差$\sigma^2$相同
  - 每个样本中的个体相互独立
2.统计量—F值
<img src="img/f.png">
<img src="img/f1.png">
 - 误差平方和SSE
 - 组间平方和SSB
 - 自由度
   - 分子:k-1
   - 分母:n-k
3. 案例
<img src="img/f2.png">

In [4]:
# 1. 计算各组的均值
x=np.array([5,5,4,4,3,5,4,4,3,3,4,3,2,2,1])
age=np.array([5,5,5,5,5,8,8,8,8,8,12,12,12,12,12])
df = pd.DataFrame({"age":age, "x":x},dtype=int)
df_group = df.groupby(["age"],as_index=True).mean()
df_group

,x
age,
5,4.2
8,3.8
12,2.4


In [28]:
# 2. 计算总体均值
means = df["x"].mean()

3.466666666666667

$∑
​i=1
​k
​​ ∑
​j=1
​n
​i
​​ 
​​ (X
​ij
​​ −
​X
​i
​​ 
​¯
​​ )
​2
​​ $

In [109]:
# 3. 计算误差平方和SSe, 这部分是每组内部的差异，即是不可解释的差异。
SSE_5 = np.sum(np.square((df.loc[df["age"]==5,'x']-df_group.loc[5,'x'])))
SSE_8 = np.sum(np.square((df.loc[df["age"]==8,'x']-df_group.loc[8,'x'])))
SSE_12 = np.sum(np.square((df.loc[df["age"]==12,'x']-df_group.loc[12,'x'])))

In [110]:
SSE = SSE_12+SSE_5+SSE_8
SSE

10.8

4. 计算组间平方和SSb，这部分是组间的，因此是可以解释的差异。
$\sum_{i=1}^{k}{n_i(\bar{X_{i}}-\bar{X})^2}$

In [113]:
SSB_5= 5 *(df_group.loc[5,'x']-means)**2
SSB_8= 5 *(df_group.loc[8,'x']-means)**2
SSB_12= 5 *(df_group.loc[12,'x']-means)**2
SSB = SSB_12+SSB_5+SSB_8
SSB

8.933333333333335

In [120]:
# 5. 计算均方误MSe,自由度n-k
k = 3
n = df.shape[0]
df1 = k-1
df2 = n-k
MSe = SSE/df2
# 6. 计算组间均方MSb,自由度 k-1 
MSb = SSB/df1

In [117]:
MSe, MSb

(0.9, 4.466666666666668)

In [121]:
# 7. 计算F， 
F = MSb/MSe

In [124]:
# 8. 得到p值
p = 1- ss.f.cdf(F, df1,df2)
p # p<0.05 Reject H0. 

0.026874464601159942

- scipy实现:

api:scipy.stats.f_oneway(*args):

- 参数:sample1, sample2, ... : array_like

- 返回值:
  - tstats:统计量值
  - pvalue:p值

In [126]:
#数据分组
df5=df[df['age']==5]['x']
df8=df[df['age']==8]['x']
df12=df[df['age']==12]['x']

In [127]:
#方差分析
f,p=ss.f_oneway(df5,df8,df12)
f,p

(4.962962962962963, 0.026874464601159935)

- statsmodels实现:

能够输出方差分析表,更优雅

In [130]:
#statsmodels实现
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
#分组字段必须是字符串类型
df.loc[:,'age']=df['age'].astype(np.str_)
#最小二乘法拟合
model = ols('x ~ age',df).fit()
#到处方差分析结果
anovat = anova_lm(model)
anovat

,df,sum_sq,mean_sq,F,PR(>F)
age,2.0,8.933333,4.466667,4.962963,0.026874
Residual,12.0,10.800000,0.900000,NaN,NaN


- 4. 事后检验
在判定组均值之间有显著差异后,仍有一些问题悬而未决.这个结果只能表明至少有两个组之间的均值有显著差异,但没有说明究竟哪几个组均值显著不同.我们必须进行事后检验

Tukey HSDTukeyHSD—TukeyTukey检验

<img src = "img/after_test1.png">

查询对应的统计量(t分布)表来判定两个组均值是否显著差异

<img src = "img/at2.png">
<img src = "img/at3.png">
<img src = "img/at4.png">

In [5]:
from statsmodels.stats.multicomp import MultiComparison
mc = MultiComparison(df['x'],df['age'])
tukey_result = mc.tukeyhsd(alpha = 0.05)
print(tukey_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     5      8     -0.4 0.7758 -1.9999  1.1999  False
     5     12     -1.8 0.0276 -3.3999 -0.2001   True
     8     12     -1.4  0.089 -2.9999  0.1999  False
----------------------------------------------------


5. 总结

- 单因子方差分析:用来判断由一个名义变量分割的多个总体的均值是不是相等
- 统计量:
$\Large F=\frac{MS_b}{MS_e}$
- 意义是分组变量解释的误差/不可解释误差 的比例,该值越大,说明分组变量影响越大
  - SST:总误差
  - SSB:组间误差(分组变量可解释的误差)
  - SSE:残差(不可解释的误差)
- api:
  - scipy: scipy.stats.f_oneway(*args)
  - statsmodels:
  - ols('x ~ age',data).fit()最小二乘拟合
  - anova_lm(model)输出方差分析表
- 事后检验:
  - 作用:找出有差异的组